In [1]:
import torch
import torch.nn as nn
from torch.optim import SGD 
import numpy as np

# Упражнение, для реализации "Ванильной" RNN
* Попробуем обучить сеть восстанавливать слово hello по первой букве. т.е. построим charecter-level модель

In [2]:
a = torch.ones((3,3))*3
b = torch.ones((3,3))*5

In [3]:
a @ b

tensor([[45., 45., 45.],
        [45., 45., 45.],
        [45., 45., 45.]])

In [4]:
a * b

tensor([[15., 15., 15.],
        [15., 15., 15.],
        [15., 15., 15.]])

In [5]:
word = 'ololoasdasddqweqw123456789'
# word = 'hello'

## Датасет. 
Позволяет:
* Закодировать символ при помощи one-hot
* Делать итератор по слову, которыей возвращает текущий символ и следующий как таргет

In [6]:
class WordDataSet:
    
    def __init__(self, word):
        self.chars2idx = {}
        self.indexs  = []
        for c in word: 
            if c not in self.chars2idx:
                self.chars2idx[c] = len(self.chars2idx)
                
            self.indexs.append(self.chars2idx[c])
            
        #print(self.chars2idx)
        #print(self.indexs)
            
        self.vec_size = len(self.chars2idx)
        self.seq_len  = len(word)
        
    def get_one_hot(self, idx):
        x = torch.zeros(self.vec_size)
        x[idx] = 1
        return x
    
    def __iter__(self):
        return zip(self.indexs[:-1], self.indexs[1:])
    
    def __len__(self):
        return self.seq_len
    
    def get_char_by_id(self, id):
        for c, i in self.chars2idx.items():
            if id == i: return c
        return None

## Реализация базовой RNN
<br/>
Скрытый элемент
$$ h_t= tanh⁡ (W_{ℎℎ} h_{t−1}+W_{xh} x_t) $$
Выход сети

$$ y_t = W_{hy} h_t $$

In [7]:
class VanillaRNN(nn.Module):
    
    def __init__(self, in_size=5, hidden_size=3, out_size=5):
        super(VanillaRNN, self).__init__()        
        
        self.n_a = hidden_size
        self.n_x = in_size
        self.n_y = out_size
        
        self.x2hidden    = nn.Linear(in_features=in_size, out_features=hidden_size)
        self.hidden      = nn.Linear(in_features=hidden_size, out_features=hidden_size)
        self.activation  = nn.Tanh()
        self.outweight   = nn.Linear(in_features=hidden_size, out_features=out_size)
    
    def forward(self, x, prev_hidden):
        hidden = self.activation(self.x2hidden(x) + self.hidden(prev_hidden))
#         Версия без активации - может происходить gradient exploding
#         hidden = self.x2hidden(x) + self.hidden(prev_hidden)
        output = self.outweight(hidden)
        return output, hidden

## Инициализация переменных 

In [8]:
ds = WordDataSet(word=word)
rnn = VanillaRNN(in_size=ds.vec_size, hidden_size=3, out_size=ds.vec_size)

# Обучение

In [9]:
def train(net, lr=0.1, n_epochs=100, CLIP_GRAD=True, max_norm=5):
    optim  = SGD(net.parameters(), lr, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(n_epochs):
        hh = torch.zeros(net.n_a)
        loss = 0
        optim.zero_grad()
        for sample, next_sample in ds:
            x = ds.get_one_hot(sample).unsqueeze(0)
            target =  torch.LongTensor([next_sample])
            y, hh = net(x, hh)
            loss += criterion(y, target)
            
        loss.backward()
    
        if epoch % 10 == 0:
            print (loss.data.item())
            if CLIP_GRAD: print("Clip gradient : ", torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm))
        else: 
            if CLIP_GRAD: torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm)
    
        optim.step()

In [10]:
train(rnn, lr=0.01, n_epochs=150)

71.97434997558594
Clip gradient :  4.90425165620005
66.48231506347656
Clip gradient :  3.4030055180284444
56.529151916503906
Clip gradient :  4.989967228275313
43.1971435546875
Clip gradient :  3.721540013347095
34.71894454956055
Clip gradient :  2.9675023548192274
29.280670166015625
Clip gradient :  2.3556675618455754
25.39620590209961
Clip gradient :  2.0376077116542572
22.33237075805664
Clip gradient :  1.643868302690176
20.0447940826416
Clip gradient :  1.393899643400129
18.324787139892578
Clip gradient :  1.192528132150409
16.956768035888672
Clip gradient :  1.0766319716431738
15.828237533569336
Clip gradient :  0.9820271809442367
14.87269401550293
Clip gradient :  0.9156732085553738
14.031759262084961
Clip gradient :  0.8774152740335416
13.260930061340332
Clip gradient :  0.8548772382027506


# Тестирование

In [11]:
rnn.eval()
hh = torch.zeros(rnn.hidden.in_features)
id = 0
softmax  = nn.Softmax(dim=1)
predword = ds.get_char_by_id(id)
for c in enumerate(word[:-1]):
    x = ds.get_one_hot(id).unsqueeze(0)
    y, hh = rnn(x, hh)
    y = softmax(y)
    m, id = torch.max(y, 1)
    id = id.data[0]
    predword += ds.get_char_by_id(id)
print ('Prediction:\t' , predword)
print("Original:\t", word)
assert(predword == word)

Prediction:	 ololoasdasdasdasdasdasdasd
Original:	 ololoasdasddqweqw123456789


AssertionError: 

# ДЗ
Реализовать LSTM и GRU модули, обучить их предсказывать тестовое слово

In [50]:
#тестовое слово
word = """«Мой дядя самых честных правил"""

## Реализовать LSTM

In [51]:
class LSTM_cell(nn.Module):
    def __init__(self, in_size, hidden_size, out_size):
        super(LSTM_cell, self).__init__()
    
        self.n_a = hidden_size
        self.n_x = in_size
        self.n_y = out_size
        
        self.forget_gate = nn.Sequential(nn.Linear(in_size + hidden_size, hidden_size), nn.Sigmoid())
        self.update_gate = nn.Sequential(nn.Linear(in_size + hidden_size, hidden_size), nn.Sigmoid())
        self.output_gate = nn.Sequential(nn.Linear(in_size + hidden_size, hidden_size), nn.Sigmoid())
        self.cand_cell  = nn.Sequential(nn.Linear(in_size + hidden_size, hidden_size), nn.Tanh())
        self.out_weight = nn.Linear(hidden_size, out_size)
        
        self.hidden_activation = nn.Tanh()
        
    def forward(self, x, prev_hidden):
        concat = torch.cat((prev_hidden.squeeze(0), x.squeeze(0)))
        
        ft = self.forget_gate(concat)
        it = self.update_gate(concat)
        ot = self.output_gate(concat)
        
        cct = self.cand_cell(concat)
        c_next = cct * it + prev_hidden * ft
        a_next = ot * self.hidden_activation(c_next)
        
        output = self.out_weight(a_next).unsqueeze(0)
        return output, a_next
    

In [52]:
ds = WordDataSet(word=word)
lstm_rnn = LSTM_cell(in_size=ds.vec_size, hidden_size=100, out_size=ds.vec_size)

In [56]:
train(lstm_rnn, lr=0.05, n_epochs=50)

80.4643325805664
Clip gradient :  1.9301206902269576
65.22058868408203
Clip gradient :  4.429612787912155
12.46749210357666
Clip gradient :  7.177492812955112
1.421187400817871
Clip gradient :  1.1585327417243547
0.0865945816040039
Clip gradient :  0.23801389310893414


In [57]:
def evaluate(net):
    net = net.eval()
    hh = torch.zeros(net.n_a)
    id = 0
    
    softmax  = nn.Softmax(dim=1)
    predword = ds.get_char_by_id(id)
    for c in enumerate(word[:-1]):
        x = ds.get_one_hot(id).unsqueeze(0)
        y, hh = net(x, hh)
        y = softmax(y)
        m, id = torch.max(y, 1)
        id = id.data[0]
        predword += ds.get_char_by_id(id)
    print ('Prediction:\t' , predword)
    print("Original:\t", word)
    assert(predword == word)

In [58]:
evaluate(lstm_rnn)

Prediction:	 «Мой дядя самых честных правил
Original:	 «Мой дядя самых честных правил


## Реализовать GRU

In [18]:
class GRU_cell(nn.Module):
    def __init__(self, in_size, hidden_size, out_size):
        super(GRU_cell, self).__init__()
    
        self.n_a = hidden_size
        self.n_x = in_size
        self.n_y = out_size
        
        self.update_gate = nn.Sequential(nn.Linear(in_size + hidden_size, hidden_size), nn.Sigmoid())
        self.relevance_gate = nn.Sequential(nn.Linear(in_size + hidden_size, hidden_size), nn.Sigmoid())
        self.candidate_cell  = nn.Sequential(nn.Linear(in_size + hidden_size, hidden_size), nn.Tanh())
        self.out_weight = nn.Linear(hidden_size, out_size)
        
        self.hidden_activation = nn.Tanh()
        self.candiate_activation = nn.Tanh()
        
    def forward(self, x, prev_hidden):
        a_prev = prev_hidden.squeeze(0)
        concat = torch.cat((a_prev, x.squeeze(0)))
        
        update_gate = self.update_gate(concat)
        relevance_gate = self.relevance_gate(concat)
        rel_candidate = relevance_gate * a_prev
        
        concat = torch.cat((rel_candidate, x.squeeze(0)))
        candidate = self.candidate_cell(concat)
        
        c_next = candidate * (1 - update_gate) + update_gate * a_prev
        output = self.out_weight(c_next).unsqueeze(0)
        return output, c_next

In [93]:
ds = WordDataSet(word=word)
gru_rnn = GRU_cell(in_size=ds.vec_size, hidden_size=5, out_size=ds.vec_size)

In [94]:
train(gru_rnn, lr=0.1, n_epochs=300)

70.4467544555664
Clip gradient :  3.59123330296043
37.14460754394531
Clip gradient :  6.717894136574477
9.919986724853516
Clip gradient :  2.7723068490420117
4.218854904174805
Clip gradient :  1.3942994671346953
3.0123491287231445
Clip gradient :  0.35867810774022196
1.8831167221069336
Clip gradient :  1.1508352155338477
0.9722824096679688
Clip gradient :  1.7333411894432598
0.49472999572753906
Clip gradient :  2.33619816195852
0.23067092895507812
Clip gradient :  0.16941317919227789
0.17565250396728516
Clip gradient :  0.22392842901428078
0.14081573486328125
Clip gradient :  0.2662560214656703
0.11734867095947266
Clip gradient :  0.19379205172769318
0.10230827331542969
Clip gradient :  0.03776777667089957
0.0935659408569336
Clip gradient :  0.05944661273579925
0.08646011352539062
Clip gradient :  0.045075577506114795
0.08057785034179688
Clip gradient :  0.022684133052044284
0.07571601867675781
Clip gradient :  0.024728948231942816
0.07147502899169922
Clip gradient :  0.019297511609911

In [95]:
evaluate(gru_rnn)

Prediction:	 ololoasdasddqweqw123456789
Original:	 ololoasdasddqweqw123456789
